# Assignment 2 question 2
Grayson Hineline

Note for portfolio: This assignment was conducted when I was very new to Python and the Pandas library. Given my current skills I would have conducted the assignment much more efficiently.

## Question 2: Find out how often Bellingham was overcast in each month of 2020.

Note from Grayson: This assignment was conducted when I was very new to Python and the Pandas library. Given my current skills I would have conducted the assignment much more efficiently.

In [1]:
import pandas as pd
import numpy as np

First we have to get the Bellingham data set into our notebook

In [2]:
bham = pd.read_csv("WA_Bellingham.csv", low_memory = False)

In [3]:
bham.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11348 entries, 0 to 11347
Data columns (total 124 columns):
 #    Column                                        Dtype  
---   ------                                        -----  
 0    STATION                                       int64  
 1    DATE                                          object 
 2    REPORT_TYPE                                   object 
 3    SOURCE                                        int64  
 4    AWND                                          float64
 5    BackupDirection                               float64
 6    BackupDistance                                float64
 7    BackupDistanceUnit                            float64
 8    BackupElements                                float64
 9    BackupElevation                               float64
 10   BackupElevationUnit                           float64
 11   BackupEquipment                               float64
 12   BackupLatitude                              

Looking at the documentation, it seems like the "HourlySkyConditions" column will give us what we want.

In [4]:
bham[["DATE", "HourlySkyConditions"]]

,DATE,HourlySkyConditions
0,2020-01-01T00:14:00,BKN:07 29
1,2020-01-01T00:24:00,SCT:04 30 SCT:04 85
2,2020-01-01T00:53:00,FEW:02 28 SCT:04 34 BKN:07 95
3,2020-01-01T01:53:00,CLR:00
4,2020-01-01T02:53:00,SCT:04 90 BKN:07 110
...,...,...
11343,2020-12-31T20:53:00,NaN
11344,2020-12-31T21:53:00,NaN
11345,2020-12-31T22:53:00,NaN
11346,2020-12-31T23:53:00,NaN


According to the documentation, if it was overcast at the time of an observation, then the three letter code before any of these numbers in the hourly sky conditions column should be 'OVC'. So to find out how often Bellingham was overcast in 2020, lets just apply the same methodology we used in question 1 to count the number of hours in which it was reported to be overcast at any time within that hour.

First we convert the date to datetime format

In [5]:
bham["DATE"] = pd.to_datetime(bham["DATE"])
bham.head(3)

,STATION,DATE,REPORT_TYPE,SOURCE,AWND,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,...,ShortDurationPrecipitationValue060,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue120,ShortDurationPrecipitationValue150,ShortDurationPrecipitationValue180,Sunrise,Sunset,TStorms,WindEquipmentChangeDate
0,72797624217,2020-01-01 00:14:00,FM-16,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72797624217,2020-01-01 00:24:00,FM-16,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72797624217,2020-01-01 00:53:00,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Again, seems like there are multiple observations per hour, so we have to implement a loop that only counts hours in which it was overcast at all once.

First let's just do the month of January to make sure I changed the values right from the code in question 1 (Changing `Boston` to `bham`, changing `HourlyPresentWeatherType` to `HourlySkyConditions`, and changing `.str.contains("RA")` to `.str.contains("OVC")`). As a refresher, this loop simply looks through each individual hour in the data set, goes to the HourlySkyConditions column, and counts whether the data for that hour contains "OVC". If it does, then that hour is counted in the total `count`.

In [6]:
count = 0
for day in range(1, 32):
    for hour in range(0,24):
        mask = (bham["DATE"] >= pd.Timestamp(2020, 1, day, hour, 0)) & (bham["DATE"] < pd.Timestamp(2020, 1, day, hour, 59))
        bhamhour = bham[mask]
        hour_mask = bhamhour["HourlySkyConditions"].str.contains("OVC")
        if(True in hour_mask.values):
            count = count + 1
    
count

521

Seemed to work, lets do the full year:

In [7]:
count = 0
for month in range(1, 13):
    length_of_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_index = month - 1
    for day in range(1, length_of_month[month_index]+1):
        for hour in range(0,24):
            mask = (bham["DATE"] >= pd.Timestamp(2020, month, day, hour, 0)) & (bham["DATE"] < pd.Timestamp(2020, month, day, hour, 59))
            bhamhour = bham[mask]
            hour_mask = bhamhour["HourlySkyConditions"].str.contains("OVC")
            if(True in hour_mask.values):
                count = count + 1
    
count

3632

Holy crap! **It was reported to be overcast at least once in 3632 hours out of the year in Bellingham in 2020.**

In [8]:
#overcast hours / total hours in the leap year
3632 / (24 * 366)

0.4134790528233151

Therefore, it was overcast at least once during 41.34% of the hours of the year in 2020.

The main question though is "How often Bellingham was overcast in each month of 2020", so we have to do this by month instead. We'll have the loop iterate, through each of the months of the year, and at the end of each iteration it will add the count of hours in which overcast was reported to a list `overcast_counts_list` in the order of the months of the year (for example, the first value the list will be January and the last will be December).

In [9]:
overcast_counts_list = []
for month in range(1, 13):
    count = 0
    length_of_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_index = month - 1
    for day in range(1, length_of_month[month_index]+1):
        for hour in range(0,24):
            mask = (bham["DATE"] >= pd.Timestamp(2020, month, day, hour, 0)) & (bham["DATE"] < pd.Timestamp(2020, month, day, hour, 59))
            bhamhour = bham[mask]
            hour_mask = bhamhour["HourlySkyConditions"].str.contains("OVC")
            if(True in hour_mask.values):
                count = count + 1
    overcast_counts_list.append(count)
    
overcast_counts_list

[521, 363, 320, 252, 232, 269, 190, 126, 238, 347, 401, 373]

Let's just sum the values in the list to confirm it counted the same amount of total hours in which overcast was reported as the earlier loop.

In [10]:
sum(overcast_counts_list)

3632

It was. Now I'm going to add that total to `overcast_counts_list` so we can have a total row in our final dataframe.

In [11]:
overcast_counts_list.append(sum(overcast_counts_list))

In [12]:
overcast_counts_list

[521, 363, 320, 252, 232, 269, 190, 126, 238, 347, 401, 373, 3632]

Now lets make this data neat, in percentages, and in a dataframe for easy viewing:

Checking percent hours of overcast for the month of January to make sure the following loop's values are correct.

In [13]:
521 / (length_of_month[0] * 24)

0.7002688172043011

This loop will make a list of the percent of hours in which it was reported overcast in 2020 (with total percent of hours overcast at the end).

In [14]:
#added 366 because thats the total days in the leap year
length_of_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 366] 
hours_of_month = []
overcast_percents = [] #This will be overcast percent by month
for i in range (0,len(length_of_month)):
    month_hours = length_of_month[i] * 24
    hours_of_month.append(month_hours)
    overcast_percents.append(overcast_counts_list[i]/month_hours)

overcast_percents

[0.7002688172043011,
 0.521551724137931,
 0.43010752688172044,
 0.35,
 0.3118279569892473,
 0.3736111111111111,
 0.2553763440860215,
 0.1693548387096774,
 0.33055555555555555,
 0.46639784946236557,
 0.5569444444444445,
 0.5013440860215054,
 0.4134790528233151]

Putting all of the data into a single dataframe.

In [15]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'Totals']
months

['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December',
 'Totals']

In [16]:
overcast = pd.DataFrame(overcast_counts_list, index = months)
overcast

,0
January,521
February,363
March,320
April,252
May,232
June,269
July,190
August,126
September,238
October,347


In [17]:
overcast["Hours in Month"] = hours_of_month
overcast

,0,Hours in Month
January,521,744
February,363,696
March,320,744
April,252,720
May,232,744
June,269,720
July,190,744
August,126,744
September,238,720
October,347,744


In [18]:
overcast["Percent Hours with any Overcast"] = overcast_percents
overcast

,0,Hours in Month,Percent Hours with any Overcast
January,521,744,0.700269
February,363,696,0.521552
March,320,744,0.430108
April,252,720,0.350000
May,232,744,0.311828
June,269,720,0.373611
July,190,744,0.255376
August,126,744,0.169355
September,238,720,0.330556
October,347,744,0.466398


In [19]:
overcast.rename(columns = {0 : "Hours with any Overcast"}, inplace = True)
overcast

,Hours with any Overcast,Hours in Month,Percent Hours with any Overcast
January,521,744,0.700269
February,363,696,0.521552
March,320,744,0.430108
April,252,720,0.350000
May,232,744,0.311828
June,269,720,0.373611
July,190,744,0.255376
August,126,744,0.169355
September,238,720,0.330556
October,347,744,0.466398


In [20]:
overcast.sort_values(by = "Percent Hours with any Overcast")

,Hours with any Overcast,Hours in Month,Percent Hours with any Overcast
August,126,744,0.169355
July,190,744,0.255376
May,232,744,0.311828
September,238,720,0.330556
April,252,720,0.350000
June,269,720,0.373611
Totals,3632,8784,0.413479
March,320,744,0.430108
October,347,744,0.466398
December,373,744,0.501344


There you have it, how often it was reported to be overcast (per hour) for all the hours of each month in 2020. It was overcast at all during January the most, and August the least.